In [23]:
def mensagem_ao_usuario(message):
    import tkinter as tk

    # create a pop-up window
    popup = tk.Tk()

    popup.title("Ação do usuario")

    # create a label with the message text

    label = tk.Label(popup, text=message)
    label.pack(padx=80, pady=20)

    # create a button to close the window
    def close_popup():
        popup.destroy()
    button = tk.Button(popup, text="OK", command=close_popup)
    button.pack(padx=80, pady=20)

    # center the window on the screen
    windowWidth = popup.winfo_reqwidth()
    windowHeight = popup.winfo_reqheight()
    positionRight = int(popup.winfo_screenwidth()/2 - windowWidth/2)
    positionDown = int(popup.winfo_screenheight()/2 - windowHeight/2)
    popup.geometry("+{}+{}".format(positionRight, positionDown))

    # run the event loop
    popup.mainloop()

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
import time
import csv

# create a new Chrome browser instance
driver = webdriver.Firefox()

# navigate to the website
driver.get("https://connect.appen.com/qrp/core/vendors/yukon_home")

# wait for the page to load
time.sleep(5)

# enter login and password
login = driver.find_element(By.XPATH, '//*[@id="username"]')
login.send_keys("engenharia.kira@gmail.com")
login = driver.find_element(By.XPATH, '//*[@id="password"]')
login.send_keys("Gfd0306?")

click_element_entrar = driver.find_element(By.XPATH, '//*[@id="kc-login"]')
click_element_entrar.click()

# wait for the page to load
time.sleep(5)

message = 'Abre a pagina do invoice e clica no invoice do mês'
mensagem_ao_usuario(message)

In [22]:
from datetime import datetime

def fill_invoice(file_name, driver):
    # Lendo o arquivo CSV com as informações
    with open(file_name, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        data = [row for row in reader]

    for i, row in enumerate(data):
        # Clicar no botão "Add New Row"
        driver.find_element(By.NAME, 'addRow').click()
        time.sleep(5)

        # Encontrar a última linha adicionada
        rows = driver.find_elements(By.CSS_SELECTOR, '.invoice-entry')
        last_row = rows[-1]

        # Converter a data para o formato mm/dd/yyyy
        date_obj = datetime.strptime(row['date'], '%Y-%m-%d')
        formatted_date = date_obj.strftime('%m/%d/%Y')

        # Selecionar Type "Regular"
        type_select = Select(last_row.find_element(By.CSS_SELECTOR, f'select[name="entries[{i}].type"]'))
        type_select.select_by_visible_text('Regular')

        # Selecionar Project "Yukon"
        project_select = Select(last_row.find_element(By.CSS_SELECTOR, f'select[name="entries[{i}].projectId"]'))
        project_select.select_by_visible_text('Yukon')

        # Preencher campos da última linha
        last_row.find_element(By.CSS_SELECTOR, f'input[name="entries[{i}].date"]').send_keys(formatted_date)

        # Escolher opções do campo "Time Worked"
        hours_select = Select(last_row.find_element(By.CSS_SELECTOR, f'select[name="entries[{i}].hours"]'))
        hours_select.select_by_value(row['elapsed_time_hours'])

        minutes_select = Select(last_row.find_element(By.CSS_SELECTOR, f'select[name="entries[{i}].minutes"]'))
        minutes_select.select_by_value(row['elapsed_time_minutes'])




        time.sleep(2)  # Aguarde um pouco antes de adicionar a próxima linha

    # Submeter o formulário
    # ...



In [ ]:
fill_invoice('converted_task_log.csv', driver)
